### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

H5_dic = read_data_header()
chunk_size = 512*30

males = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'
print males.length(), 'elements'
print fem.length(), 'elements'


Data with 40 male and female speakers
4397 elements
4382 elements


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model configuration

In [4]:
N = 256
max_pool = 128
batch_size = 1
learning_rate = 0.001

config_model = {}
config_model["type"] = "pretraining"

config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size

config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["window"] = 1024

config_model["smooth_size"] = 20

config_model["alpha"] = learning_rate
config_model["reg"] = 1e-4
config_model["beta"] = 0.1
config_model["rho"] = 0.05



# Model creation

In [ ]:
from models.adapt import Adapt

adapt_model = Adapt(config_model=config_model, pretraining=True, folder='pretraining')
print 'Total name :' 
print adapt_model.runID
adapt_model.tensorboard_init()

ID : AdaptiveNet-rapid-water-2147
[<tf.Variable 'conv/W:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'deconv/WT:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'smooth/smoothing_filter:0' shape=(1, 20, 256, 256) dtype=float32_ref>]
Total name :
AdaptiveNet-rapid-water-2147-N=256--alpha=0.001--batch_size=1--beta=0.1--chunk_size=15360--maxpool=128--reg=0.0001--rho=0.05--smooth_size=20--type=pretraining--window=1024-


# Pretraining the model 

In [ ]:
nb_iterations = 1000

#initialize the model
adapt_model.init()


for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = adapt_model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        adapt_model.save(i)

Step # 0  loss= 0.759443
DAS model saved at iteration number  0  with cost =  0.759443
Step # 1  loss= 0.694683
Step # 2  loss= 0.640291
Step # 3  loss= 0.631551
Step # 4  loss= 0.475101
Step # 5  loss= 0.392115
Step # 6  loss= 0.593143
Step # 7  loss= 0.481935
Step # 8  loss= 0.65757
Step # 9  loss= 0.329286
Step # 10  loss= 0.451102
Step # 11  loss= 0.599127
Step # 12  loss= 0.377243
Step # 13  loss= 0.189822
Step # 14  loss= 0.186707
Step # 15  loss= 0.393727
Step # 16  loss= 0.277342
Step # 17  loss= 0.33335
Step # 18  loss= 0.339616
Step # 19  loss= 0.332901
Step # 20  loss= 0.23364
DAS model saved at iteration number  20  with cost =  0.23364
Step # 21  loss= 0.26682
Step # 22  loss= 0.228428
Step # 23  loss= 0.288753
Step # 24  loss= 0.284649
Step # 25  loss= 0.292417
Step # 26  loss= 0.403939
Step # 27  loss= 0.219317
Step # 28  loss= 0.375505
Step # 29  loss= 0.165311
Step # 30  loss= 0.249726
Step # 31  loss= 0.433522
Step # 32  loss= 0.245869
Step # 33  loss= 0.216888
Step #